In [29]:
import pandas as pd
import gzip as gz
import sqlite3
from zipfile import ZipFile

In [30]:
with gz.open('data/tn.movie_budgets.csv.gz') as f:
    budgets = pd.read_csv(f,encoding='latin1')

budgets

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [31]:
imdb = 'data/im.db.zip'
with ZipFile(imdb, 'r') as zip:
    zip.extractall('data/imdb_unzipped')

conn = sqlite3.connect('data/imdb_unzipped/im.db')

In [32]:
q = """
SELECT pr.movie_id, pr.person_id, pr.category, p.primary_name, m.primary_title AS 'Movie Title'
FROM principals pr
    JOIN persons p
        USING(person_id)
    JOIN movie_basics m
        USING(movie_id)
"""
principals_df = pd.read_sql(q, conn)

In [33]:
principals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1027912 entries, 0 to 1027911
Data columns (total 5 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   movie_id      1027912 non-null  object
 1   person_id     1027912 non-null  object
 2   category      1027912 non-null  object
 3   primary_name  1027912 non-null  object
 4   Movie Title   1027912 non-null  object
dtypes: object(5)
memory usage: 39.2+ MB


In [34]:
# Remove extraneous '$' and ',' symbols
budgets['production_budget'] = budgets['production_budget'].str.replace(',', '')
budgets['production_budget'] = budgets['production_budget'].str.replace("$", '')

budgets['domestic_gross'] = budgets['domestic_gross'].str.replace(",", '')
budgets['domestic_gross'] = budgets['domestic_gross'].str.replace("$", '')

budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace(",", '')
budgets['worldwide_gross'] = budgets['worldwide_gross'].str.replace("$", '')

In [35]:
# Convert money columns to integers
cash_columns = ['production_budget', 'domestic_gross', 'worldwide_gross']
budgets[cash_columns] = budgets[cash_columns].apply(pd.to_numeric)

In [36]:
# Convert to datetime
budgets['release_date'] = pd.to_datetime(budgets['release_date'])

In [37]:
# Create column to evaluate profitability
budgets['net_earnings'] = budgets['worldwide_gross'] - budgets['domestic_gross']
budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_earnings
0,1,2009-12-18,Avatar,425000000,760507625,2776345279,2015837654
1,2,2011-05-20,Pirates of the Caribbean: On Stranger Tides,410600000,241063875,1045663875,804600000
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000
3,4,2015-05-01,Avengers: Age of Ultron,330600000,459005868,1403013963,944008095
4,5,2017-12-15,Star Wars Ep. VIII: The Last Jedi,317000000,620181382,1316721747,696540365


In [38]:
# Filter budgets to include only those in our target range
recent = budgets.loc[(budgets['release_date'] >= '2012-01-01')]

# Verify earliest entry
recent['release_date'].min()        

Timestamp('2012-01-06 00:00:00')

In [42]:
recent.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1666 entries, 2 to 5780
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 1666 non-null   int64         
 1   release_date       1666 non-null   datetime64[ns]
 2   movie              1666 non-null   object        
 3   production_budget  1666 non-null   int64         
 4   domestic_gross     1666 non-null   int64         
 5   worldwide_gross    1666 non-null   int64         
 6   net_earnings       1666 non-null   int64         
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 104.1+ KB


In [39]:
# Merge recent and principals

combined = recent.merge(principals_df, left_on='movie', right_on='Movie Title', how='inner')
combined

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,net_earnings,movie_id,person_id,category,primary_name,Movie Title
0,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000,tt6565702,nm0498278,writer,Stan Lee,Dark Phoenix
1,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000,tt6565702,nm0564215,actor,James McAvoy,Dark Phoenix
2,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000,tt6565702,nm1055413,actor,Michael Fassbender,Dark Phoenix
3,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000,tt6565702,nm2225369,actress,Jennifer Lawrence,Dark Phoenix
4,3,2019-06-07,Dark Phoenix,350000000,42762350,149762350,107000000,tt6565702,nm0396558,actor,Nicholas Hoult,Dark Phoenix
...,...,...,...,...,...,...,...,...,...,...,...,...
18514,81,2015-09-29,A Plague So Pleasant,1400,0,0,0,tt2107644,nm4572412,actress,Eva Boehnke,A Plague So Pleasant
18515,81,2015-09-29,A Plague So Pleasant,1400,0,0,0,tt2107644,nm4767160,actor,David Chandler,A Plague So Pleasant
18516,81,2015-09-29,A Plague So Pleasant,1400,0,0,0,tt2107644,nm5497357,actor,Maxwell Moody,A Plague So Pleasant
18517,81,2015-09-29,A Plague So Pleasant,1400,0,0,0,tt2107644,nm4766951,director,Benjamin Roberds,A Plague So Pleasant


In [50]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18519 entries, 0 to 18518
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 18519 non-null  int64         
 1   release_date       18519 non-null  datetime64[ns]
 2   movie              18519 non-null  object        
 3   production_budget  18519 non-null  int64         
 4   domestic_gross     18519 non-null  int64         
 5   worldwide_gross    18519 non-null  int64         
 6   net_earnings       18519 non-null  int64         
 7   movie_id           18519 non-null  object        
 8   person_id          18519 non-null  object        
 9   category           18519 non-null  object        
 10  primary_name       18519 non-null  object        
 11  Movie Title        18519 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(6)
memory usage: 1.8+ MB


In [49]:
individual = combined.groupby('primary_name')['net_earnings'].mean()
individual

primary_name
50 Cent                  3.980041e+07
A. Scott Berg            5.904238e+06
A.A. Milne               9.828972e+07
A.D. Oppenheim           2.994550e+06
A.J. DeLucia             0.000000e+00
                             ...     
Étienne Sauret           0.000000e+00
Ólafur Darri Ólafsson    1.034513e+08
Óscar Jaenada            6.364182e+07
Óskar Jónasson           3.187886e+07
Ömür Atay                1.720000e+07
Name: net_earnings, Length: 13033, dtype: float64